In [1]:
import numpy as np
from load_new import getData
import xgboost as xgb
import pandas as pd
from sklearn import preprocessing
from sklearn import cross_validation
import pickle
import os.path
import scipy
import random

trX,trY,teX,teY = getData(oh=0)

dtrain = xgb.DMatrix(trX, label = trY)
dtest = xgb.DMatrix(teX, label = teY)

hyperparams_grid = {
    'n_round' : 800,
    'early_stopping' : 100,
    'bst:max_depth' : scipy.stats.randint(10,30),
    'bst:eta' : scipy.stats.uniform(0.01,0.2),
    'bst:subsample' : scipy.stats.uniform(0.5,0.5),
    'bst:colsample_bytree' : scipy.stats.uniform(0.7,0.3)
    }
hyperparams_grid

                            

    
def draw(v):
    if hasattr(v, 'rvs'):
        return v.rvs()
    elif type(v) is list:
        return random.choice(v)
    else:
        return v


2224
3106
536
759
450

load_new.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  temp[:,j]=np.fft.fft(temp[:,j])
load_new.py:85: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  teX=trX[:cut,:]


load_new.py:86: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  teY=trY[:cut]
load_new.py:87: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  trX=trX[cut:,:]
load_new.py:88: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  trY=trY[cut:]


In [2]:
def evaluate_hyperparams(hyperparams):
    def get_model_results(mdl):
            res = {}
            res['ntrees'] = mdl.best_iteration+1
            res['score'] = mdl.best_score
            res['best_ntrees'] = mdl.best_iteration+1
            res['best_score'] = mdl.best_score
            return res
    def get_results_df():
        if os.path.isfile('results/results_df_last.pkl'):
            return pd.io.pickle.read_pickle('results/results_df_last.pkl')
        else:
            return pd.DataFrame()

    n_round = hyperparams['n_round']
    early_stopping = hyperparams['early_stopping']
    plst = [
            ('bst:max_depth', hyperparams['bst:max_depth']),
            ('objective', 'multi:softprob'),
            ('silent', 1),
            ('bst:eta', hyperparams['bst:eta']),
            ('bst:subsample', hyperparams['bst:subsample']),
            ('bst:colsample_bytree', hyperparams['bst:colsample_bytree']),
            ('num_class', 4),
    ]
    evallist  = [(dtrain,'train'), (dtest,'test')]
    mdl = xgb.train(plst, dtrain, 
                    num_boost_round = n_round, 
                    evals = evallist,
                    early_stopping_rounds = early_stopping)
    line_dict = get_model_results(mdl)
    line_dict.update(hyperparams)
    line_dict = {k:[v] for k,v in line_dict.iteritems()}
    line_df = pd.DataFrame(line_dict)
    results_df = get_results_df()
    results_df = results_df.append(line_df, ignore_index = True)
    results_df.to_pickle('results/results_df_last.pkl')
    results_df.to_pickle('results/old/results_df_%s.pkl' % (len(results_df)-1))
    file = open("results/results_last.txt", "w")
    file.write(str(results_df))
    file.close()
    file = open("results/old/results_%s.txt" % (len(results_df)-1), "w")
    file.write(str(results_df))
    file.close()

In [3]:
N = 1

for k in xrange(N):
    print '--------------------------'
    print 'Evaluating hyperparameters'
    print k
    hp = {k:draw(v) for k, v in hyperparams_grid.iteritems()}
    print hp
    evaluate_hyperparams(hp)

Will train until test error hasn't decreased in 100 rounds.
[0]	train-merror:0.073780	test-merror:0.121439
[1]	train-merror:0.052066	test-merror:0.105091
[2]	train-merror:0.040859	test-merror:0.103690
[3]	train-merror:0.040159	test-merror:0.096684
[4]	train-merror:0.036657	test-merror:0.089211
[5]	train-merror:0.035022	test-merror:0.086408
[6]	train-merror:0.034789	test-merror:0.085941
[7]	train-merror:0.031520	test-merror:0.086875
[8]	train-merror:0.028251	test-merror:0.085474
[9]	train-merror:0.028018	test-merror:0.083139
[10]	train-merror:0.027784	test-merror:0.081738
[11]	train-merror:0.027084	test-merror:0.082205
[12]	train-merror:0.026850	test-merror:0.080803
[13]	train-merror:0.026850	test-merror:0.080336
[14]	train-merror:0.024282	test-merror:0.080336
[15]	train-merror:0.023582	test-merror:0.080336
[16]	train-merror:0.021947	test-merror:0.078001
[17]	train-merror:0.021013	test-merror:0.076133
[18]	train-merror:0.019846	test-merror:0.077067
[19]	train-merror:0.018912	test-merror

--------------------------
Evaluating hyperparameters
0
{'bst:colsample_bytree': 0.7939438211106268, 'bst:subsample': 0.5161884514083115, 'bst:max_depth': 12, 'early_stopping': 100, 'n_round': 800, 'bst:eta': 0.1073408314176849}


Stopping. Best iteration:
[154]	train-merror:0.000000	test-merror:0.060252

